In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install swig --quiet
# !pip install gymnasium --quiet
!pip install stable_baselines3 --quiet
!pip install gymnasium[box2d] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [8]:
%cd '/content/drive/MyDrive/CSE_251b/251b_final_project/Dima_colabs/adaptive_award/'

/content/drive/.shortcut-targets-by-id/1RCyQOI1i_gXpHt3VOJnq2dG0-KSqMU10/251b_final_project/Dima_colabs/adaptive_award


In [3]:
import gymnasium as gym
from gymnasium.envs.registration import register
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

import os
import sys
sys.path.append('/content/drive/MyDrive/CSE_251b/251b_final_project/Dima_colabs/adaptive_award/')


save_dir = '/content/drive/MyDrive/CSE_251b/251b_final_project/Dima_colabs/adaptive_award/'
# model_dir = os.path.join(save_dir, 'car_racing_ppo3')
# curr_model_dir = os.path.join(save_dir, 'car_racing_ppo_curr')
save_folder = os.path.join(save_dir, 'car_saves')
os.makedirs(save_folder, exist_ok=True)
best_model_dir = os.path.join(save_folder, 'best_model.zip')


In [4]:
register(
    id='CustomCarRacing-v0',  # Use a new ID for your custom environment
    # entry_point='custom2_car_racing:CustomCarRacing',
    entry_point='adaptive_race_car_wrapper:CustomCarRacing',
    max_episode_steps=1000,
    reward_threshold=1000,
)

register(
    id='CustomCarRacing-v2',  # Use a new ID for your custom environment
    # entry_point='custom2_car_racing:CustomCarRacing',
    entry_point='adaptive_race_car_wrapper:Late_game_CustomCarRacing',
    max_episode_steps=1000,
    reward_threshold=1000,
)

transer_model = False
trasfer_model_dir = os.path.join(save_dir, 'car_saves/best_model.zip')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Create and wrap the environment
env = gym.make('CustomCarRacing-v0')
env = Monitor(env)  # Wrap the environment with the Monitor wrapper
env = DummyVecEnv([lambda: env])  # Then wrap it in a dummy vectorized environment



# Initialize the agent
model = PPO("CnnPolicy", env, verbose=0)
if transer_model:
    model.load(trasfer_model_dir, env=env)


total_iterations = 50
time_steps = 5000
best_mean_reward = -float('inf')

old_reward =0
iter = 0
for iteration in range(total_iterations):
    iter += 1
    model_dir = os.path.join(save_folder, f'model_{iteration + 1}')
    # curr_model_dir = os.path.join(save_folder, f'model_{iteration + 1}_curr')
    # print(f"Training epoch: {iteration + 1}/{total_iterations}")
    model.learn(total_timesteps=time_steps, reset_num_timesteps=False)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

    if mean_reward > 400 and old_reward > 400:
        print("changing to late game")
        break

    old_reward = mean_reward
    print(f"Iteration: {iteration + 1}, Mean reward: {mean_reward}, Std reward: {std_reward}")

    # Save the current model
    model.save(model_dir)
    if mean_reward > best_mean_reward:
        best_mean_reward = mean_reward
        model.save(best_model_dir)
        print(f"Model saved at iteration {iteration + 1}")





/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CustomCarRacing-v0 is out of date. You should consider upgrading to version `v2`.
  logger.deprecation(


Iteration: 1, Mean reward: -86.2450161, Std reward: 4.9189027165521875
Model saved at iteration 1
Iteration: 2, Mean reward: -344.73435340000003, Std reward: 27.09300701739682
Iteration: 3, Mean reward: 110.90436340000001, Std reward: 150.45208539004915
Model saved at iteration 3
Iteration: 4, Mean reward: -77.51801789999999, Std reward: 86.8284283501387
Iteration: 5, Mean reward: -120.50066450000001, Std reward: 151.22913402574247
Iteration: 6, Mean reward: -149.98357259999997, Std reward: 117.0394937263736
Iteration: 7, Mean reward: -47.561959300000005, Std reward: 82.60997678408316
Iteration: 8, Mean reward: -66.36828829999999, Std reward: 178.70366198344615
Iteration: 9, Mean reward: -165.57224100000002, Std reward: 118.29022119436009
Iteration: 10, Mean reward: -50.03907929999998, Std reward: 99.6043026004035
Iteration: 11, Mean reward: -42.6263859, Std reward: 91.80563052186393
Iteration: 12, Mean reward: -77.16864149999999, Std reward: 143.08179237145663
Iteration: 13, Mean rewa

In [5]:
# iter =20

In [6]:
env2 = gym.make('CustomCarRacing-v2')
env2 = Monitor(env2)  # Wrap the environment with the Monitor wrapper
env2 = DummyVecEnv([lambda: env2])  # Then wrap it in a dummy vectorized environment

model2 = PPO("CnnPolicy", env2, verbose=0)
model_dir = os.path.join(save_folder, f'model_{iter}')
model2 = PPO.load(model_dir, env=env2)

total_iterations = 50
time_steps = 5000
best_mean_reward = -float('inf')

for iteration in range(iter, total_iterations):
  model_dir = os.path.join(save_folder, f'model_{iteration + 1}')
  model2.learn(total_timesteps=time_steps, reset_num_timesteps=False)
  mean_reward, std_reward = evaluate_policy(model2, model2.get_env(), n_eval_episodes=10)

  print(f"Iteration: {iteration + 1}, Mean reward: {mean_reward}, Std reward: {std_reward}")

  # Save the current model
  model2.save(model_dir)
  if mean_reward > best_mean_reward:
      best_mean_reward = mean_reward
      model2.save(best_model_dir)
      print(f"Model saved at iteration {iteration + 1}")






Iteration: 21, Mean reward: 829.4441569999999, Std reward: 237.7384201748185
Model saved at iteration 21
Iteration: 22, Mean reward: 622.8134680000001, Std reward: 253.0890989010702
Iteration: 23, Mean reward: 835.9653394000001, Std reward: 158.00632387903667
Model saved at iteration 23
Iteration: 24, Mean reward: 459.53804049999997, Std reward: 306.0742284183068
Iteration: 25, Mean reward: 444.30533429999997, Std reward: 223.88738505960234
Iteration: 26, Mean reward: 602.8535470999999, Std reward: 168.04918866162882
Iteration: 27, Mean reward: 302.3845195999999, Std reward: 183.32256318834516
Iteration: 28, Mean reward: 349.3380635, Std reward: 219.17268521947423
Iteration: 29, Mean reward: 201.8208325, Std reward: 139.64070217045602
Iteration: 30, Mean reward: 271.5722472, Std reward: 185.8370101260925
Iteration: 31, Mean reward: 467.9468718, Std reward: 241.8836870119369
Iteration: 32, Mean reward: 589.5661837, Std reward: 208.34193274943325
Iteration: 33, Mean reward: 755.9538773, 

KeyboardInterrupt: 